In [536]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import nltk
import nltk as nlp
import string
import re
import spacy
from spacy import displacy
from spacy.util import minibatch, compounding

In [537]:
td1 = pd.read_excel("TD1L.xlsx")
td2 = pd.read_excel("TD2L.xlsx")
td3 = pd.read_excel("TD3L.xlsx")
cde = pd.read_excel("CDE.xlsx")
ts = pd.read_excel("TS.xlsx")

In [538]:
[td1.shape, td2.shape, td3.shape]

[(5865, 40), (5865, 36), (5865, 23)]

In [539]:
TD = pd.merge(td1, td2, on='ID', how = 'left')

In [540]:
TD

,ID,CONTROL,VIV_SEL,HOGAR,R_SEL,DP1_1_1,DP1_4_1,DP1_4_2,DP1_4_3,DP1_4_4,...,DP4_3_6_2,DP4_3_6_3,DP4_3_6_4,DP4_3_6_5,DP4_1_7,DP4_3_7_1,DP4_3_7_2,DP4_3_7_3,DP4_3_7_4,DP4_3_7_5
0,2800407101,28004,7,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2804901102,28049,1,1,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2804501103,28045,1,1,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2800109101,28001,9,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2801805101,28018,5,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860,24001105101,240011,5,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5861,24503405101,245034,5,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5862,24503620101,245036,20,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5863,24601528101,246015,28,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [541]:
TDL = pd.merge(TD, td3, on='ID', how='right')

In [542]:
TDT = pd.merge(TDL, ts, on='ID', how='right')

In [543]:
TDT.fillna(9.9,inplace=True)
TDT

,ID,CONTROL_x,VIV_SEL_x,HOGAR_x,R_SEL,DP1_1_1,DP1_4_1,DP1_4_2,DP1_4_3,DP1_4_4,...,GRA,P2_8,P2_9,P2_10,FAC_VIV,FAC_ELE_y,DOM_y,EST_y,UPM_y,CIUDAD_y
0,2800407101,28004.0,7.0,1.0,1.0,9.9,9.9,9.9,9.9,9.9,...,5.0,1.0,9.9,4.0,75,88,U,302,4950,44.0
1,2804901102,28049.0,1.0,1.0,2.0,9.9,9.9,9.9,9.9,9.9,...,5.0,5.0,4.0,9.9,122,242,U,301,5400,44.0
2,2804501103,28045.0,1.0,1.0,3.0,9.9,9.9,9.9,9.9,9.9,...,3.0,1.0,9.9,2.0,127,595,U,302,5360,44.0
3,2800109101,28001.0,9.0,1.0,1.0,9.9,9.9,9.9,9.9,9.9,...,2.0,1.0,9.9,1.0,151,291,U,302,4920,44.0
4,2801805101,28018.0,5.0,1.0,1.0,9.9,9.9,9.9,9.9,9.9,...,3.0,5.0,4.0,9.9,127,209,U,302,5090,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232052,24028705101,9.9,9.9,9.9,9.9,9.9,9.9,9.9,9.9,9.9,...,3.0,6.0,4.0,9.9,550,0,U,197,67250,9.9
232053,24028705102,9.9,9.9,9.9,9.9,9.9,9.9,9.9,9.9,9.9,...,1.0,5.0,4.0,9.9,550,0,U,197,67250,9.9
232054,24028705103,9.9,9.9,9.9,9.9,9.9,9.9,9.9,9.9,9.9,...,1.0,4.0,4.0,9.9,550,0,U,197,67250,9.9
232055,24028705104,9.9,9.9,9.9,9.9,9.9,9.9,9.9,9.9,9.9,...,1.0,4.0,4.0,9.9,550,2149,U,197,67250,9.9


In [544]:
x = cde.Clave
y = cde.Entidad
dicc = {x[i]:y[i] for i in range(len(x))}

In [414]:
a=TDL
b=TDL
c=TDL
d=TDL
e=TDL
x = [a,b,c,d,e]
for v in TDL.columns:
    i=0
    if v[:5]=='DP1_4':
        x[i+1] = pd.merge(x[i], cde, left_on= v, right_on='Clave', how='right')
        x[i+1].drop([v, 'Clave'], axis='columns', inplace=True)
        x[i+1].rename(columns={'Entidad':v},inplace=True)
        i+=1

In [545]:
df.f = [dicc.get(n, n) for n in TDT.DP1_4_1]
df.f

['Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Baja California',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Baja California',
 'Baja California',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Baja California',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Baja California',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Baja California',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Sin Delito',
 'Distrito Federal',
 'Si

In [546]:
import pandas as pd
from pandas import ExcelWriter

In [547]:
writer = ExcelWriter('TDT.xlsx')
TDT.to_excel(writer, 'Hoja de datos', index=False)
writer.save()